In [6]:
import pandas as pd
import numpy as np
import re

In [7]:
# 패턴을 이용하여 저장 값을 리스트로 되돌리는 함수
def make_ls(c,i): # 인덱스 리스트와 순서를 받음
    matches = re.findall(r"'(.*?)'", c[i])
    return list(matches)

def clear_text(sentence):  # 목차 내용 정제 함수
    pattern = r'[\w가-힣]+'
    extracted = re.findall(pattern, sentence)
    result = ' '.join(extracted)
    return result
    
def delete_text(sentence):
    pattern = r'u3000|u000|u00|uf0ab|u2005'
    result = re.sub(pattern, '', sentence)
    return result
    
def split_pn(line): # 페이지 분리 함수
    a = re.sub(r'[^(X|IX|IV|V?I|Ⅱ|Ⅲ)]','', line)
    if line.split(' ')[-1].isnumeric() == True and a == '':
        page = line.split(' ')[-1]  
        line = ' '.join(line.split(' ')[:-1])
#         line = line.replace(line.split(' ')[-1]  ,'').strip() # 멀쩡한 애도 지워서
    else:
        page = np.NaN
    return line, page

# 총론 각론 인식 해결
def plus_pandan(pd_l):
    if '총' in pd_l and '각' in pd_l:
        i = pd_l.index('총')
        pd_l[i] = ['총', '각']
        pd_l.remove('각')
    elif '총론' in pd_l and '각론' in pd_l:
        i = pd_l.index('총론')
        pd_l[i] = ['총론', '각론']
        pd_l.remove('각론')
#     elif '총론' in pd_l and '각론' in pd_l and '부록' in pd_l:
#         i = pd_l.index('총론')
#         pd_l[i] = ['총론', '각론', '부록']
    elif '형법총론' in pd_l and '형법각론' in pd_l:
        i = pd_l.index('형법총론')
        pd_l[i] = ['형법총론', '형법각론']
        pd_l.remove('형법각론')
    elif '헌법총론' in pd_l and '헌법각론' in pd_l:
        i = pd_l.index('헌법총론')
        pd_l[i] = ['헌법총론', '헌법각론']
        pd_l.remove('헌법각론')
    else:
        pass
    return pd_l

In [8]:
# 등록 함수 정의
def clear_idx(isbn, title, text, i, ex):
    code = isbn[i] # isbn
    title = title[i] # main_title
    if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
        text.remove('목차 더보기') 
    text = [clear_text(r) for r in text] # 이상한 문자 제거
    text = [delete_text(r) for r in text] # 특정 문자 제거
    pandan, roma, ara= classification_list2(text, ex) # 분류 판단 기준 생성 함수
    pandan = plus_pandan(pandan)
    print('clearidx',pandan)
    if 'check' in pandan:
        isbn_ls.append(code)
        main_title.append(title)
        etc.append('check')
        for ls in [depth_1, depth_2, depth_3, depth_4, pages]:
            ls.append(np.NaN)
    elif 'check' not in pandan:
        if len(pandan) == 1:
            for num, line in enumerate(text):
                l,p = split_pn(line) # 페이지 번호랑 line 분류
                depth_1.append(l)
                depth_2.append(np.NaN)
                depth_3.append(np.NaN)
                depth_4.append(np.NaN)
                etc.append(0) # 문제 없음
                pages.append(p)
                isbn_ls.append(code)
                main_title.append(title)
        elif len(pandan) == 2:
            for num, line in enumerate(text):
                a = re.sub(r'[^(Ⅹ|Ⅸ|Ⅳ|Ⅴ?Ⅰ|Ⅱ|Ⅲ)]','', line) 
                l,p = split_pn(line)
                if l[0].isnumeric() == True and l[1] in ['부', '절','편','강']: # 숫자분류
                    check = 'num'
                elif l[0].isnumeric() == True  and l[1] not in ['부', '절','편','강'] and a == '': # 아라비아
                    check = 'ara'
                elif l[0].isnumeric() == True and a != '': # roma
                    check = 'roma'
                elif l[0].isnumeric() != True:
                    b = re.sub(r'[^a-zA-Z가-힣]','',line.lower())
                    b = ex.match(b)
                    if b:
                        check = b.group()
                    else:
                        print(num,'적용 X')
                store_new(pandan, check, l, p, code, title) # store 수정중
        elif len(pandan) == 3:
            for num, line in enumerate(text):
                a = re.sub(r'[^(Ⅹ|Ⅸ|Ⅳ|Ⅴ?Ⅰ|Ⅱ|Ⅲ)]','', line)
                l,p = split_pn(line)
                if l[0].isnumeric() == True and l[1] in ['부', '절','편','강']: # 숫자분류
                    check = 'num'
                elif l[0].isnumeric() == True  and l[1] not in ['부', '절','편','강'] and a == '': # 아라비
                    check = 'ara'
                elif l[0].isnumeric() == True and a != '': # roma
                    check = 'roma'
                elif l[0].isnumeric() != True:
                    b = re.sub(r'[^a-zA-Z가-힣]','',line.lower())
                    b = ex.match(b)
                    
                    if b:
                        print(num, b.group())
                        check = b.group()
                        print(pandan)
                    else:
                        print(num,'적용 X')
                store_new(pandan, check, l, p, code, title) # store 수정중
                        
        elif len(pandan) >= 4:
            for num, line in enumerate(text):
                a = re.sub(r'[^(Ⅹ|Ⅸ|Ⅳ|Ⅴ?Ⅰ|Ⅱ|Ⅲ)]','', line)
                l,p = split_pn(line)
                if l[0].isnumeric() == True and l[1] in ['부', '절','편','강']: # 숫자분류
                    check = 'num'
                elif l[0].isnumeric() == True  and l[1] not in ['부', '절','편','강'] and a == '': # 아라비
                    check = 'ara'
                elif l[0].isnumeric() == True and a != '': # roma
                    check = 'roma'
                elif l[0].isnumeric() != True:
                    b = re.sub(r'[^a-zA-Z가-힣]','',line.lower())
                    b = ex.match(b)
                    if b:
                        check = b.group()
#                         store_new(pandan, check, l, p, code, title) # store 수정중
                    else:
                        print(num,'적용 X')
                store_new(pandan, check, l, p, code, title) # store 수정중

In [9]:
def classification_list2(text, ex): # 판별 요소 설정 함수
    pd_l,roma_l, ara_l = [],[],[]
    for idx, line in enumerate(text):
        a = re.sub(r'[^(X|IX|IV|V?I|Ⅱ|Ⅲ)]','', line)
        # 숫자 판별
        if line[0].isnumeric() == True: 
            if a != '' and 'roma' not in pd_l: # 로마 숫자
                roma_l.append(idx) # 위치 저장
                pd_l.append('roma')
            elif a != '' and 'roma' in pd_l: # 로마 숫자
                roma_l.append(idx) # 위치 저장
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' not in pd_l :
                ara_l.append(idx)
                pd_l.append('num') # 분류랑 숫자 결합
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' in pd_l:
#                 ara_l.append(idx)
                print('num')
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' not in pd_l :
                ara_l.append(idx)
                pd_l.append('ara') # 분류랑 숫자 결합
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' in pd_l:
                ara_l.append(idx)
        elif line[0].isnumeric() != True:   # 포함되는 리스트
            t = re.sub(r'[^a-zA-Z가-힣]','',line.lower())  # 기록 시에는 line을 기록해야 함
            b = ex.match(t)
            if b and b.group() not in pd_l:
                pd_l.append(b.group())
#                 print(b.group(), text)
            elif b and b.group() in pd_l:
                pass
            else:
                pd_l.append('check')
                print(t)      
                
    return pd_l, roma_l, ara_l

In [10]:
def store_new(pandan, check, l, p, code, title):    
    print('check',check)
#     l = delete_text(l)  ################################ 오류 발생시 주석 풀어서 테스트
    if type(pandan[0]) == list:
        try:
            print('store_new',pandan[0].index(check))
            depth_dict[f'{0}'].append(l)
            for ls in [1,2,3]:
                depth_dict[f'{ls}'].append(np.NaN)
            etc.append(e)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
        except:
            ii = pandan.index(check)
            if ii >= 3:
                ii = 3
            else:
                ii = ii
            depth_dict[f'{ii}'].append(l)
            check_ls = [0,1,2,3]
            check_ls.remove(ii)
            for ls in check_ls:
                
                depth_dict[f'{ls}'].append(np.NaN)
            etc.append(e)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
    elif type(pandan[1]) == list:  # 이부분 추가됨
        try:
            print(pandan[1].index(check))
            depth_dict[f'{1}'].append(l)
            for ls in [0,2,3]:
                depth_dict[f'{ls}'].append(np.NaN)
            etc.append(e)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
        except:
            ii = pandan.index(check)
            if ii >= 3:
                ii = 3
            else:
                ii = ii
            depth_dict[f'{ii}'].append(l)
            check_ls = [0,1,2,3]
            check_ls.remove(ii)
            for ls in check_ls:
                depth_dict[f'{ls}'].append(np.NaN)
            etc.append(e)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
    else:
        ii = pandan.index(check)
        if ii >= 3:
            ii = 3
        else:
            ii = ii
        depth_dict[f'{ii}'].append(l)
        check_ls = [0,1,2,3]
        check_ls.remove(ii)
        for ls in check_ls:
            depth_dict[f'{ls}'].append(np.NaN)
        etc.append(e)
        pages.append(p)
        isbn_ls.append(code)
        main_title.append(title)

In [11]:
ex_t = re.compile('제절|제편|제강|제항|제장|제회|제부|제과|제관|사례|형법총론|헌법총론|형법각론|헌법각론|부록|키워드|정답|해설|차례|참고|판례|핵심|part|chapter|day|section|theme|case|keyword|point|question|contents|강|목|장|항|회|부|총론|각론|q')

In [12]:
ex_q = re.compile('최신|기출|모의|제회|제부|제과|제관|사례|형법총론|헌법총론|형법각론|헌법각론|부록|키워드|문제|정답|해설|차례|참고|판례|핵심|part|chapter|day|section|theme|case|keyword|point|question|contents|강|목|장|항|회|부|총론|각론|q')

In [13]:
data = pd.read_excel("C:\police_data\police_book_idx_type.xlsx")

# 변수 설정
isbn, title, text,book_t = data['isbn'], data['교재명'], data['index'],data['책유형']


isbn_ls, main_title, depth_1, depth_2, depth_3, depth_4, etc, pages = [],[],[],[],[],[],[], []
depth_dict = {'0':depth_1,'1':depth_2,'2':depth_3,'3':depth_4}

In [14]:
depth_dict = {'0':depth_1,'1':depth_2,'2':depth_3,'3':depth_4}

# 실행
for i in range(len(text)):
    try:
        tet = make_ls(text, i)
        if book_t[i] == 'T':
            clear_idx(isbn, title, tet, i, ex_t)
        elif book_t[i] =='Q':
            clear_idx(isbn, title, tet, i, ex_q)
#         clear_idx2(a,b,text,i)
        print(i)
    except:
        print('i')
        isbn_ls.append(isbn[i])
        main_title.append(title[i])
        etc.append(2)  # 데이터가 존재하지 않는 경우
        for k in [depth_1, depth_2,depth_3,depth_4,pages]:
            k.append(np.NaN)

제편헌법총론
제장헌법과헌법학
제절헌법의의의
제항헌법의개념과분류
제항헌법의특성
제절헌법학연구
제항헌법의해석
제항헌법관
제절헌법의제정
제절헌법의개정
제절헌법의변천
제절헌법의수호
제장대한민국헌법총설
제절대한민국헌정사
제절대한민국의국가형태
제절대한민국의구성요소
제절한국헌법의기본원리
제항한국헌법의전문
제항국민주권의원리
제항민주주의의원리
제항법치주의의원리
제항사회국가의원리
제항문화국가의원리
제항평화국가의원리
제절정당제도
제절선거제도
제절공무원제도
제절지방자치제도
제편기본권론
제장기본권총론
제절기본권보장의역사
제절기본권의성격
제절기본권의주체
제절기본권의포기
제절기본권의효력
제절기본권의보호의무
제절기본권의갈등
제절기본권의제한과그한계
제절기본권의보호
제장인간의존엄과가치행복추구권평등권
제절인간의존엄과가치행복추구권
제절평등권
제장자유권적기본권
제절인신에관한자유
제항생명권
제항신체를훼손당하지않을권리
제항신체의자유
제절사생활영역의자유
제항사생활의비밀과자유
제항주거의자유
제항거주이전의자유
제항통신의자유
제절정신생활영역의자유
제항양심의자유
제항종교의자유
제항언론출판의자유
제항집회결사의자유
제항학문예술의자유
제절경제생활영역의자유
제항재산권
제항직업의자유
제항소비자의권리
제장정치적기본권
제절정치적기본권의의의와유형
제절참정권
제장청구권적기본권
제절청구권적기본권의의의및유형
제절청원권
제절재판청구권
제절국가배상청구권
제절형사보상청구권
제절범죄피해자구조청구권
제장사회적기본권
제절사회적기본권일반론
제절인간다운생활을할권리
제절교육을받을권리
제절근로의권리
제절근로권
제절환경권
제절혼인과가족생활의보장
제절모성의보호와보건권
제장국민의기본적의무
clearidx ['check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check', 'check'

In [15]:
df_ls = pd.DataFrame({'isbn':isbn_ls, 'main_title':main_title,'depth_1':depth_1, 'depth_2':depth_2,\
                      'depth_3':depth_3, 'depth_4': depth_4,'pages':pages,'etc':etc})
df_ls.to_excel('C:/police_data/06_27_test5.xlsx',encoding = 'utf-8-sig', index=False) 

ValueError: All arrays must be of the same length

In [17]:
len(isbn), len(depth_1)

(169, 384)

# 연ㅅ,ㅂ

In [80]:

# 실행
for i in range(len(text)):
    try:
        tet = make_ls(text, i)
        print(tet)
        print('_  _  _')
        for t in tet:
            print(len(tet))
            t = clear_text(t)
            t = delete_text(t)
            print('---')
            key, knum = count_words(t)
            print(i, type(key),type(knum))
    except:
        print('i')
#         isbn_ls.append(isbn[i])
#         main_title.append(title[i])
#         etc.append(2)  # 데이터가 존재하지 않는 경우
#         for k in [depth_1, depth_2,depth_3,depth_4,pages]:
#             k.append(np.NaN)

['/제1편/ 헌법총론', '제1장 헌법과 헌법학 2', '제1절 헌법의 의의 2', '제1항 헌법의 개념과 분류 3', '제2항 헌법의 특성 6', '제2절 헌법학 연구 8', '제1항 헌법의 해석 8', '제2항 헌법관 12', '제3절 헌법의 제정 12', '제4절 헌법의 개정 13', '제5절 헌법의 변천 20', '제6절 헌법의 수호 20', '제2장 대한민국헌법총설 24', '제1절 대한민국헌정사 24', '제2절 대한민국의 국가형태 32', '제3절 대한민국의 구성요소 33', '제4절 한국헌법의 기본원리 46', '제1항 한국헌법의 전문 46', '제2항 국민주권의 원리 51', '제3항 민주주의의 원리 53', '제4항 법치주의의 원리 55', '제5항 사회국가의 원리 70', '제6항 문화국가의 원리 77', '제7항 평화국가의 원리 79', '제5절 정당제도 83', '제6절 선거제도 92', '제7절 공무원제도 102', '제8절 지방자치제도 105', '/제2편/ 기본권론', '제1장 기본권 총론 112', '제1절 기본권보장의 역사 112', '제2절 기본권의 성격 113', '제3절 기본권의 주체 115', '제4절 기본권의 포기 124', '제5절 기본권의 효력 124', '제6절 기본권의 보호의무 126', '제7절 기본권의 갈등 131', '제8절 기본권의 제한과 그 한계 136', '제9절 기본권의 보호 149', '제2장 인간의 존엄과 가치ㆍ행복추구권ㆍ평등권 151', '제1절 인간의 존엄과 가치ㆍ행복추구권 151', '제2절 평등권 164', '제3장 자유권적 기본권 176', '제1절 인신에 관한 자유 176', '제1항 생명권 176', '제2항 신체를 훼손당하지 않을 권리 178', '제3항 신체의 자유 179', '제2절 사생활영역의 자유 217', '제1항 사생활의 비밀과 자유 217', '제2항 주거의 자유 228', '제3항 거주ㆍ이전의 자유 229', '제4항 통신의 자유 231', '제3절 정신생활영역의 자

57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class 'int'>
145
---
57 <class 'str'> <class '

72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class 'int'>
449
---
72 <class 'str'> <class '

95 <class 'str'> <class 'int'>
107
---
95 <class 'str'> <class 'int'>
107
---
95 <class 'str'> <class 'int'>
107
---
95 <class 'str'> <class 'int'>
107
---
95 <class 'str'> <class 'int'>
107
---
2word 1
95 <class 'str'> <class 'int'>
['[1권]', 'PART Ⅰ 헌법 일반이론', 'CHAPTER 01 헌법과 헌법학', 'CHAPTER 02 헌법의 변동과 헌법의 보호', 'CHAPTER 03 대한민국', 'CHAPTER 04 대한민국헌법의 기본원리', 'CHAPTER 05 국민주권주의와 자유민주주의', 'CHAPTER 06 법치주의', 'CHAPTER 07 대통령의 권한', 'CHAPTER 08 사회·경제·문화적 기본원리', 'CHAPTER 09 국제질서의 기본원리', 'CHAPTER 10 헌법상 제도', 'CHAPTER 11 정당제도와 정당설립의 자유', 'CHAPTER 12 선거권과 선거제도', 'CHAPTER 13 직업공무원제와 공무담임권', 'CHAPTER 14 지방자체제도', '[2권]', 'PART Ⅱ 국민의 권리와 의무', 'CHAPTER 01 기본권 일반이론', 'CHAPTER 02 포괄적 기본권', 'CHAPTER 03 평등원칙과 평등권', 'CHAPTER 04 인신의 자유', 'CHAPTER 05 사생활영역의 보호', 'CHAPTER 06 정신적 자유권', 'CHAPTER 07 경제적 기본권', 'CHAPTER 08 정치적 기본권 (참정권)', 'CHAPTER 09 사회적 기본권', 'CHAPTER 10 청구권', 'CHAPTER 11 국민의 기본의무', '목차 더보기']
_  _  _
30
---
96 <class 'int'> <class 'int'>
30
---
96 <class 'str'> <class 'int'>
30
---
96 <class 'str'>

134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 'int'>
82
---
134 <class 'str'> <class 